### **Generating all possible solutions for [Mathler](https://mathler) game** *([Wordle](https://www.nytimes.com/games/wordle/index.html)'s mathematic variant)*

First of all, I thought this wouldn't be feasable, that the ammount of possible equations and results would be to large to compute. But I wanted to try it anyways, and it turns out, there aren't that many valid possible equations.

Let's list the explicit rules for each possible solution:
+ Each equation must be exactly 6 chars long *(each char is considered to be a single digit number or an operator)*
+ Numbers and operators can appear more than once
+ No leading zeros are allowed
+ Calculations follow the order of operations **PEDMAS**
    1. Parentheses
    2. Exponents
    3. Division or Multiplication
    4. Addition or Subtraction

We can also extrapolate some other rules based on the previous requirements:

+ Equations can start with a `'-'` sign, but can't start with any other operator *(`'/','*','+'`)*
+ Equations can't end with any of the operators *(`'/','*','+','-'`)*
+ Only divisions that result in an integer are valid
+ Divisions by zero are not allowed
+ There can't be equations that are equal to the solution:
  + 6 digit numbers
  + 5 digit negative numbers

Besides all of that, there are 3 difficulty levels:

+ **Easy Mathler** *(5 squares and 1 operator)*
+ **Mathler** *(6 squares and 2 operators)*
+ **Hard Mathler** *(8 squares and 3 operators)*

In [59]:
from tqdm import tqdm
import itertools
import csv
import re
import os

Defining constants:

In [60]:
DIRPATH = "data" # The directory path for the output files

NUMBERS = ["1", "2", "3", "4", "5", "6", "7", "8", "9", "0"]  # List of valid numbers
OPERATORS = ["/", "*", "+", "-"] # List of valid operators
NEGATIVE_OPERATORS = ["-"]  # '-' operator

EMPTY_STRING = ""  # An empty string

LEVELS = {  # Stores all off the levels of difficulty data
    "easy": {
        "filepath": "equations_easy.csv",
        "operators_count": 1,
        "pattern": [
            ["1", "2", "3", "4", "5", "6", "7", "8", "9", "0", "-"],
            ["1", "2", "3", "4", "5", "6", "7", "8", "9", "0", "/", "*", "+", "-"],
            ["1", "2", "3", "4", "5", "6", "7", "8", "9", "0", "/", "*", "+", "-"],
            ["1", "2", "3", "4", "5", "6", "7", "8", "9", "0", "/", "*", "+", "-"],
            ["1", "2", "3", "4", "5", "6", "7", "8", "9", "0"],
        ],
    },
    "int": {
        "filepath": "equations_int.csv",
        "operators_count": 2,
        "pattern": [
            ["1", "2", "3", "4", "5", "6", "7", "8", "9", "0", "-"],
            ["1", "2", "3", "4", "5", "6", "7", "8", "9", "0", "/", "*", "+", "-"],
            ["1", "2", "3", "4", "5", "6", "7", "8", "9", "0", "/", "*", "+", "-"],
            ["1", "2", "3", "4", "5", "6", "7", "8", "9", "0", "/", "*", "+", "-"],
            ["1", "2", "3", "4", "5", "6", "7", "8", "9", "0", "/", "*", "+", "-"],
            ["1", "2", "3", "4", "5", "6", "7", "8", "9", "0"],
        ],
    },
    "hard": {
        "filepath": "equations_hard.csv",
        "operators_count": 3,
        "pattern": [
            ["1", "2", "3", "4", "5", "6", "7", "8", "9", "0", "-"],
            ["1", "2", "3", "4", "5", "6", "7", "8", "9", "0", "/", "*", "+", "-"],
            ["1", "2", "3", "4", "5", "6", "7", "8", "9", "0", "/", "*", "+", "-"],
            ["1", "2", "3", "4", "5", "6", "7", "8", "9", "0", "/", "*", "+", "-"],
            ["1", "2", "3", "4", "5", "6", "7", "8", "9", "0", "/", "*", "+", "-"],
            ["1", "2", "3", "4", "5", "6", "7", "8", "9", "0", "/", "*", "+", "-"],
            ["1", "2", "3", "4", "5", "6", "7", "8", "9", "0", "/", "*", "+", "-"],
            ["1", "2", "3", "4", "5", "6", "7", "8", "9", "0"],
        ],
    },
}

Defining helper functions:

In [61]:
def delete_csv(filepath):
    """Deletes a csv file"""

    if os.path.exists(filepath):
        os.remove(filepath)


def append_csv(filepath, row):
    """Appends a row to a csv file"""

    with open(filepath, "a", newline="") as file:
        csv.writer(file).writerow(row)

Defining the equation solver function:

In [62]:
def solve(input_equation):
    """Solves an equation passed as a string"""

    # Join all consecutive numbers
    equation = []
    temp = EMPTY_STRING

    for char in input_equation:
        if char in NUMBERS:
            temp += char

        else:
            if temp != EMPTY_STRING:
                equation.append(temp)

            equation.append(char)
            temp = EMPTY_STRING

    equation.append(temp)

    # If first element is a negative operator, join it to the next element (number)
    if equation[0] == NEGATIVE_OPERATORS[0]:
        equation[1] = f"{NEGATIVE_OPERATORS[0]}{equation[1]}"
        equation.pop(0)

    # Perform all operations (following the operators order - PEDMAS)
    for i in range(0, 3, 2):
        operators = OPERATORS[i : i + 2]

        while operators[0] in equation or operators[1] in equation:
            for j, _ in enumerate(equation):

                if equation[j] in operators:
                    operator = equation[j]

                    result = eval(f"{equation[j - 1]}{operator}{equation[j + 1]}")

                    if result != int(result):
                        return None

                    equation[j - 1] = int(result)

                    for _ in range(2):
                        equation.pop(j)

                    break

    # Return equations first element
    return equation[0]

Testing the solver:

In [63]:
# Define test cases
test_equations = [
    ("-3+4/2", -1),
    ("-3*4/2", -6),
    ("-10+10", 0),
    ("10-1-9", 0),
    ("40/4/2", 5),
    ("1+2+30", 33),
    ("3+21/3", 10),
    ("40/4+2", 12),
    ("16+4*2", 24),
    ("2/1*40", 80),
    ("1+2/30", None),
    ("2/3*10", None),
]

# Iterate over equations and check if they are solved correctly
for equation, solution in test_equations:
    assert solve(equation) == solution, f"{equation} should be {solution}"

Defining the function that generates all of the valid equations for a given difficulty level:

In [64]:
def gen_equations(difficulty, levels, dirpath):
    """Generates all valid equations for the game"""

    # Extract variables and store them
    operators_count=levels[difficulty]["operators_count"]
    filepath = f"{dirpath}\\{levels[difficulty]['filepath']}"
    pattern=levels[difficulty]["pattern"]

    # Calculate total number of candidate equations
    candidates_count = 1

    for subpattern in pattern:
        candidates_count *= len(subpattern)

    # Combinations of elements for each position of the equation
    equations = itertools.product(*pattern)

    # Delete previous csv file
    delete_csv(filepath)

    # Iterate over each equation
    for equations in tqdm(equations, total=candidates_count):

        # Convert equation to a single string
        equation = "".join(equations)

        # Count number of operators in an equation
        counter = 0
        for char in equation:
            if char in OPERATORS:
                counter += 1

        if equation[0] == NEGATIVE_OPERATORS[0]:
            counter -= 1

        if counter != operators_count:
            continue
        
        # Discard equations containing consecutive operators
        if re.findall(r"[/,*,+,-][/,*,+,-]", equation):
            continue
        
        # Discard equations containing leading zeros
        if re.findall(r"0\d", equation):
            continue

        # Discard equations containing leading zeros
        if re.findall(r"/0", equation):
            continue

        # Solve the equation
        solution = solve(equation)

        # Discard equations with invalid solutions
        if not solution:
            continue

        # Discard equations that are equal to it's solution
        if equation == solution:
            continue
        
        # Check if the solution matches the equation using 'eval()'
        if eval(equation) != solution:
            print(f"The equation is different then the eval() solution: {equation} != {solution}")
            break

        # Append equation to csv file
        append_csv(filepath, [equation, solution])

Generating all valid equations for the easy difficulty level:

In [65]:
gen_equations("easy", LEVELS, DIRPATH)

100%|██████████| 301840/301840 [00:30<00:00, 9802.92it/s] 


Generating all valid equations for the intermediate difficulty level:

In [66]:
gen_equations("int", LEVELS, DIRPATH)

100%|██████████| 4225760/4225760 [02:24<00:00, 29170.28it/s] 


Generating all valid equations for the hard difficulty level:

In [68]:
gen_equations("hard", LEVELS, DIRPATH)

PermissionError: [WinError 32] The process cannot access the file because it is being used by another process: 'data\\equations_hard.csv'